# Import Statements #

In [87]:
import os
import topogenesis as tg
import pyvista as pv
import ipywidgets as widgets
import itertools
import trimesh as tm
import scipy as sc
from scipy.cluster.vq import kmeans, kmeans2, vq , whiten
import numpy as np
import random
import pandas as pd
from itertools import cycle
import skcriteria as sk
from skcriteria.agg import similarity  # here lives TOPSIS
from skcriteria.pipeline import mkpipe  # this function is for create pipelines
from skcriteria.preprocessing import invert_objectives, scalers
pv.set_jupyter_backend('trame')

#Display full array without truncation
np.set_printoptions(threshold=np.inf)

## 01. Loading the base object ##

We are loading the base .obj file here to create voxels. The voxel size is considered here is 3x3x3m and the base object is extruded to 2m so that a grid of only 1 voxel in depth is created. In the Hub implementation we can consider adding code to create a 2d voxel grid with the current 3d implementation

In [110]:
vs = 3
unit = [vs, vs, vs]
tol = 1e-09
mesh_path = os.path.relpath(r"C:\Users\aditya.soman\source\repos\Loci_Python\Obj_Data\floorobject.obj")
mesh = tg.geometry.load_mesh(mesh_path)


In [111]:
sample_cloud, ray_origins = tg.geometry.mesh_sampling(mesh, unit, multi_core_process=False, return_ray_origin = True, tol=tol)
lattice = sample_cloud.voxelate(unit, closed=True)

In [113]:
indices = np.argwhere(lattice.flatten() == 1).flatten()
centers = lattice.centroids
# initiating the plotter
p = pv.Plotter(notebook=True) # ITK plotter for interactivity within the python notebook (itkwidgets library is required)


# fast visualization of the point cloud
#sample_cloud.fast_notebook_vis(p)

# fast visualization of the lattice
lattice.fast_notebook_vis(p)

# adding the base mesh: light blue
mesh = pv.read(mesh_path)
#p.add_mesh(mesh, color='#abd8ff')

# adding the ray origins: dark blue
#p.add_points(pv.PolyData(ray_origins), color='#004887')
#p.add_points(centers, color='#ff5733')

p.add_point_labels(centers, indices, font_size=10, point_size=5)
p.view_xy()
#p.screenshot("high_res_image.png", window_size=[4000, 3000])
# plotting
p.show()


Widget(value='<iframe src="http://localhost:53400/index.html?ui=P_0x1878909f750_11&reconnect=auto" class="pyvi…

## 02. Create Various Performance matrices (Lattices in topogenesis terminology)
1. Quiteness : I have added a few points in Rhino as potential noise sources and the argmin distances from these points are considered as the level of quiteness for that voxel. Note quiteness is used and not dB levels since we are not calculating any sound insulations etc.
2. Distance from Facades : Distance from facades N,S,E,W. If the users have a priority to configure zones close to a specific facade 
3. Daylight : Simplified daylight analysis values for the voxels 
4. Distance from Lobby : It is the distance of the voxels from the Lobby entrances 
5. Distance from an Core: It is the distance of the voxels from the Core entrances 

In [91]:
class PerformanceLattice:
    def __init__(self, base_lattice):
        self.base_lattice = base_lattice
        self.north_distance = np.max(base_lattice.centroids.T[1])
        self.south_distance = np.min(base_lattice.centroids.T[1])
        self.east_distance = np.max(base_lattice.centroids.T[0])
        self.west_distance = np.min(base_lattice.centroids.T[0])

    def create_distance_lattice(self, point_lattice:np.array) -> None:
        """
        Create a distance based lattice  on the point sources and initial lattice.
        The resulting distance is arg min values from the closest point
        """
        # Create availability lattice
        init_lattice = self.base_lattice
        availability_lattice_voxels = tg.to_lattice(init_lattice, init_lattice)
        voxel_coordinates = availability_lattice_voxels.centroids

        # Flatten the initial lattice
        flattened_lattice = self.base_lattice.flatten()

        # Compute Euclidean distances
        eucledian_distance = sc.spatial.distance.cdist(point_lattice, voxel_coordinates)
        distance_from_each_source = eucledian_distance.T

        # Compute the average quiteness values
        average_distance_indexing = np.argmin(distance_from_each_source, axis=1)
        average_distance_values = [branch[index] for branch, index in zip(distance_from_each_source, average_distance_indexing)]

        # Compute the quiteness lattice
        distance_lattice_padded = np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(average_distance_values))])
        distance_lattice_np = distance_lattice_padded.reshape(self.base_lattice.shape)

        # Convert to lattice format
        distance_lattice = tg.to_lattice(distance_lattice_np, distance_lattice_np.shape)

        return distance_lattice
    
    def create_facade_lattice(self, direction:str) -> None:
        if direction == 'south':
            evaluation_value = self.north_distance
            distance_lattice = evaluation_value- self.base_lattice.centroids.T[1].reshape(self.base_lattice.shape)
        elif direction == 'north':
            evaluation_value = self.south_distance
            distance_lattice = np.abs(evaluation_value-self.base_lattice.centroids.T[1].reshape(self.base_lattice.shape))
        elif direction == 'west':
            evaluation_value = self.east_distance
            distance_lattice = evaluation_value-self.base_lattice.centroids.T[0].reshape(self.base_lattice.shape)
        elif direction == 'east':
            evaluation_value = self.west_distance
            distance_lattice = np.abs(evaluation_value-self.base_lattice.centroids.T[0].reshape(self.base_lattice.shape))
        return distance_lattice
    
    def distance_from_outermostvoxel (self):
        return self.base_lattice

In [92]:
def csv_nparray(file_name:str) -> np.array:
    """
    Convert a csv file to numpy array
    """
    Complete_file = pd.read_excel(file_name, sheet_name=0,engine='openpyxl',header = None )
    array_excel = Complete_file.to_numpy()
    point_array = array_excel.T
    return point_array
# Numpy array of points indicating source of noise
Noise_sources = csv_nparray('Large_financial_Center_Noise.xlsx')
Core_centers = csv_nparray('Core_Centers_FinancialBuilding.xlsx')
Daylight_points = csv_nparray('Facade_Origins.xlsx')
# Create Performance Lattice creator class
Performance_Creator = PerformanceLattice(lattice+1)
Quiteness_lattice = Performance_Creator.create_distance_lattice(Noise_sources)
Coredis_lattice = Performance_Creator.create_distance_lattice(Core_centers)
East_facade_lattice = Performance_Creator.create_facade_lattice('east')
West_facade_lattice = Performance_Creator.create_facade_lattice('west')
North_facade_lattice = Performance_Creator.create_facade_lattice('north')
South_facade_lattice = Performance_Creator.create_facade_lattice('south')
#Closeness to Facade can be equated to daylight availability
Daylight_simplified = Performance_Creator.create_distance_lattice(Daylight_points)


In [93]:
base_lattice = ~lattice
Quiteness_lattice[base_lattice] = 0
###### initiating the plotter
p = pv.Plotter(notebook=True)
# Create the spatial reference
grid = pv.ImageData()

# Set the grid envelope_lattice: shape because we want to inject our values
nx, ny, nz = base_lattice.shape
grid.dimensions = (nx + 1, ny + 1, nz + 1)  # Dimensions = cells + 1 for points
# The bottom left corner of the data set
grid.origin = lattice.minbound
# These are the cell sizes along each axis
grid.spacing = lattice.unit

# Add the data values to the cell data Select whichever analysis you want to display
grid.cell_data["Quiteness"] = Quiteness_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
lattice.fast_vis(p)

# adding the meshes

opacity = [0, 0.75, 0, 0.75, 1.0] 
clim = [0, 100]
p.add_volume(grid, cmap="magma", clim=clim,
             opacity=opacity, opacity_unit_distance=5,)
# Add your points to plot 
p.add_points( Noise_sources, color='#FB5607',render_points_as_spheres=True, point_size=20)

p.show()

Widget(value='<iframe src="http://localhost:53400/index.html?ui=P_0x187890a0790_8&reconnect=auto" class="pyvis…

## 03. Create Desirability lattices ( Perform MCDA on the  based on the various performance matrices developed in the previous step ) ##

In [94]:
# Collect all the Performance matrice
array_all_performance_matrices = [Daylight_simplified.flatten(),Quiteness_lattice.flatten(),Coredis_lattice.flatten(), East_facade_lattice.flatten(), West_facade_lattice.flatten(), North_facade_lattice.flatten(), South_facade_lattice.flatten()]
performance_matrix = np.array(array_all_performance_matrices).T

In [95]:
class DesirabilityLattice:
    def __init__(self, performance_matrix:np.array):
        self.performance_matrix = performance_matrix
    
    def topsis (self, objectives_array, weights_array, criteria_array):
        """
        TOPSIS method to calculate the desirability of each voxel
        """
        decision_matrix = sk.mkdm(
        self.performance_matrix,
        objectives = objectives_array,
        weights = weights_array,
        criteria = criteria_array)

        pipe = mkpipe(
        invert_objectives.NegateMinimize(),
        scalers.VectorScaler(target="matrix"),  # this scaler transform the matrix
        scalers.SumScaler(target="weights"),  # and this transform the weights
        similarity.TOPSIS(),
        )
        rank = pipe.evaluate(decision_matrix)
        mcdm_result= rank.e_.similarity.reshape(lattice.shape)
        
        return mcdm_result


In [114]:
# # Zone 1 : Staff Restaurants can be noisy 
# # Zone 2 : Executive suites should be quite and very close to the facade 
# # zone 3: Client meeting suite has to be very quite and closer to the facade 
# # Zone 4 : Landscape should be on the South side 

criteria_array = ["daylight","quietness", "coredis", "eastfacade", "westfacade", "northfacade", "southfacade"]
objectives_array = [min,max,min,max,max,max,max]
decisionmaker = DesirabilityLattice(performance_matrix)

#Zone1 parameters Staff Restaurants
Z1_weights_array = [0,0,0, 0.5, 0, 0,0.5]
Z1_Desirability_lattice = decisionmaker.topsis(objectives_array, Z1_weights_array, criteria_array)

#Zone2 parameters  Executive suites
Z2_weights_array = [0.4,0.3,0,0,0,0.3,0]
Z2_Desirability_lattice = decisionmaker.topsis(objectives_array, Z2_weights_array, criteria_array)

#Zone3 parameters  Client meeting suite
Z3_weights_array = [0.2,0.6,0,0.4,0,0.4,0]
Z3_Desirability_lattice = decisionmaker.topsis(objectives_array, Z3_weights_array, criteria_array)

#Zone4 parameters  Landscaping
Z4_weights_array = [0,0, 0, 0, 0, 0,1]
Z4_Desirability_lattice = decisionmaker.topsis(objectives_array, Z4_weights_array, criteria_array)

Z4_Desirability_lattice

array([[[1.        , 1.        , 1.        ],
        [0.97222222, 0.97222222, 0.97222222],
        [0.94444444, 0.94444444, 0.94444444],
        [0.91666667, 0.91666667, 0.91666667],
        [0.88888889, 0.88888889, 0.88888889],
        [0.86111111, 0.86111111, 0.86111111],
        [0.83333333, 0.83333333, 0.83333333],
        [0.80555556, 0.80555556, 0.80555556],
        [0.77777778, 0.77777778, 0.77777778],
        [0.75      , 0.75      , 0.75      ],
        [0.72222222, 0.72222222, 0.72222222],
        [0.69444444, 0.69444444, 0.69444444],
        [0.66666667, 0.66666667, 0.66666667],
        [0.63888889, 0.63888889, 0.63888889],
        [0.61111111, 0.61111111, 0.61111111],
        [0.58333333, 0.58333333, 0.58333333],
        [0.55555556, 0.55555556, 0.55555556],
        [0.52777778, 0.52777778, 0.52777778],
        [0.5       , 0.5       , 0.5       ],
        [0.47222222, 0.47222222, 0.47222222],
        [0.44444444, 0.44444444, 0.44444444],
        [0.41666667, 0.41666667, 0

In [97]:
# ###### initiating the plotter
# p = pv.Plotter(notebook=True)
# # Create the spatial reference
# grid = pv.ImageData()

# # Set the grid envelope_lattice: shape because we want to inject our values
# nx, ny, nz = lattice.shape
# grid.dimensions = (nx + 1, ny + 1, nz + 1)  # Dimensions = cells + 1 for points
# # The bottom left corner of the data set
# grid.origin = lattice.minbound
# # These are the cell sizes along each axis
# grid.spacing = lattice.unit

# # Add the data values to the cell data Select whichever analysis you want to display
# grid.cell_data["Client Meeting suite"] = Z3_Desirability_lattice.flatten(order="F")  # Flatten the Lattice

# # fast visualization of the lattice
# lattice.fast_vis(p)

# # adding the meshes
# # Matplot lib colormaps https://matplotlib.org/stable/users/explain/colors/colormaps.html

# opacity = [0.5, 0.75, 0.5, 0.75, 0.5] 
# clim = [0, 100]
# p.add_volume(grid, cmap="BrBG", clim=clim,
#              opacity=opacity, opacity_unit_distance=5,)
# # Add your points to plot 
# #p.add_points( Noise_sources, color='#FB5607',render_points_as_spheres=True, point_size=20)
# p.view_xy()
# p.show()

## 04. Initiate Agent classes ##

The Computational agents have certain behaviours and can take certain topological actions in the program. In the case of this notebook the agents can occupy ( claim) voxels in the floor plan for a particular zone during an instance of a simulation. The agents can navigate through the voxelated space by means of a stencil which defines the neighbourhood definition like Von-neumann neighbourhood or Moora neighbourhood (https://en.wikipedia.org/wiki/Von_Neumann_neighborhood#:~:text=The%20von%20Neumann%20neighbourhood%20of,cellular%20automaton%20in%20three%20dimensions.)

In [98]:
# creating neighborhood definition
stencil_von_neumann = tg.create_stencil("von_neumann", 1, 1)
stencil_von_neumann.set_index([0,0,0], 0)

# creating neighborhood definition
stencil_moore = tg.create_stencil("moore", 1, 1)
stencil_moore.set_index([0,0,0], 0)

# creating neighborhood definition 
stencil_squareness_moore = tg.create_stencil("moore", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness_moore[0,:,:] = 0 
stencil_squareness_moore[2,:,:] = 0
stencil_squareness_moore.set_index([0,0,0], 0)
stencil_squareness_t = np.transpose(stencil_squareness_moore) 

# creating neighborhood definition 
stencil_squareness_von = tg.create_stencil("von_neumann", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness_von[0,:,:] = 0 
stencil_squareness_von[2,:,:] = 0
stencil_squareness_von.set_index([0,0,0], 0)
stencil_squareness_von_t = np.transpose(stencil_squareness_von)


In [99]:
# Definition for an agent class
class agent():
    def __init__(self, origin, stencil, id):

        # define the origin attribute of the agent and making sure that it is an intiger
        self.origin = np.array(origin).astype(int)
        # define old origin attribute and assigning the origin to it as the initial state
        self.old_origin = self.origin
        # define stencil of the agent
        self.stencil = stencil
        #define agent id
        self.id = id
        #Number of voxels left 
        self.voxelsoccupied = 1
        
   
    def rectangularish_occupy(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        #print(self.origin)
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        #print(neighs)

        unravelled_neighs = np.array(np.unravel_index(neighs, env.availibility.shape)).T
        #print(unravelled_neighs)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]  

        
        # find env values of all neighbours
        all_neighs_value = env.value.flatten()[neighs]
        all_neighs_value_mod = np.copy(all_neighs_value)
        
        
        #finding number of neighbours and bumping the values based on adjacency for a 9 neighbourhood
        """
        This method is assuming that you will have a 9 neighbourhood everytime but it is not the case so try to addd if functions etc to make it
        work on neighbourhoods with smaller number of neighbours as well
        """

        #print(neighbourhood_details)
        one = neighs_availibility[4] + neighs_availibility[5] 
        two = neighs_availibility[4] + neighs_availibility[6] 
        three = neighs_availibility[5] + neighs_availibility[7] 
        four = neighs_availibility[6] + neighs_availibility[7] 
        five = neighs_availibility[1] + neighs_availibility[0] 
        six = neighs_availibility[0] + neighs_availibility[2] 
        seven = neighs_availibility[1] + neighs_availibility[3] 
        eight = neighs_availibility[2] + neighs_availibility[3] 
        neighbourhood_details = [one, two, three, four, five, six, seven, eight]


        #print(neighbourhood_details)
        for detail in range(len(neighs_availibility)-1):
            neighbourhood_condition = neighbourhood_details[detail] 
            #print(neighbourhood_condition)
            if neighbourhood_condition == self.id or neighbourhood_condition == self.id+1:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + 1
                #print("One neigh  found")
            elif neighbourhood_condition == self.id*2:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + 2
                #print("Two neigh  found")
            else:
                all_neighs_value_mod[detail] = all_neighs_value_mod[detail] 

        #print(all_neighs_value_mod)

        neighs_value_flattened = env.value.flatten()
        
        
        for neigh, val_mod in zip(neighs, all_neighs_value_mod):
             neighs_value_flattened[neigh] = val_mod
        
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]

        avail_index = np.array(np.where(env.availibility == 1))
        neighs_availibility_full_floor = np.ravel_multi_index(avail_index, env.availibility.shape)

        if len(free_neighs)== 0 :
            free_neighs = neighs_availibility_full_floor
            #print("Full floor considered")
        else: 
            free_neighs= free_neighs

        # retrieve the value of each neighbour
        free_neighs_value = neighs_value_flattened[free_neighs]
        #print(free_neighs)
        #print(free_neighs_value)

        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #print(selected_neigh)
        # update information
        ####################
        self.voxelsoccupied = np.sum(env.availibility.flatten()== self.id)
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)

    def squarish_occupy(self,env, iter):
        num_iter = env.number_of_iterations
        current_step = iter
        def generate_pattern(total_numbers):
            # Initialize an empty list to store the result
            pattern = []

            # We need to keep track of the number of digits added so far
            current_count = 0
            i = 1  # Start with row 1 (initially number 3)

            while current_count < total_numbers:
                if i % 2 != 0:  # Odd rows (e.g., row 1, 3, 5...)
                    row = [int(digit) for digit in '1' * i] + [int(digit) for digit in '3' * i]
                else:  # Even rows (e.g., row 2, 4, 6...)
                    row = [int(digit) for digit in '2' * i] + [int(digit) for digit in '0' * i]

                # Check if adding this row exceeds the total number of digits
                if current_count + len(row) > total_numbers:
                    row = row[:total_numbers - current_count]  # Trim the row if needed
                
                pattern.append(row)
                current_count += len(row)
                i += 1  # Increment the row number
            
            # Flatten the pattern into a 1D NumPy array
            flattened_pattern = np.concatenate(pattern)
            
            return flattened_pattern
        # Generate the flattened 1D NumPy array
        result = generate_pattern(iter+10)
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        occupy_id = neighs[result[iter]]

        neighs_availibility = env.availibility.flatten()[neighs]

        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(occupy_id, env.availibility.shape)).flatten()

        self.voxelsoccupied = np.sum(env.availibility.flatten()== self.id)
        #print(self.remainingvoxels)

        if neighs_availibility[result[iter]] == 1 :
            return True
    

In [100]:
# Definition for an enviornment class
# environment class
class environment():
    def __init__(self, lattices, agents,number_of_iterations,method_name):
        self.availibility = lattices["availibility"]
        self.value = lattices["enviornment"]
        self.agent_origin = self.availibility 
        self.agents = agents
        self.update_agents_rectangularish()
        self.number_of_iterations = number_of_iterations
        self.method_name = method_name
        self.neigh_squareness = []
        #Number of voxels left 
        self.voxelsleft = 100       

    def update_agents_rectangularish(self):
        # making previous position available
      #  self.availibility[tuple(self.agents.old_origin)] = self.availibility[tuple(self.agents.old_origin)] * 0 + 1
        # removing agent from previous position
        self.agent_origin[tuple(self.agents.old_origin)] *= 0+1
        # making the current position unavailable
        self.availibility[tuple(self.agents.origin)] = self.agents.id
        # adding agent to the new position 
        self.agent_origin[tuple(self.agents.origin)] = self.agents.id

    def update_agents_squarish(self,bool_response):
        # making previous position available
        #  self.availibility[tuple(self.agents.old_origin)] = self.availibility[tuple(self.agents.old_origin)] * 0 + 1
        # removing agent from previous position
        self.agent_origin[tuple(self.agents.old_origin)] *= 0+1
        # making the current position unavailable
        if bool_response == True :
            self.availibility[tuple(self.agents.origin)] = self.agents.id
            # adding agent to the new position 
            self.agent_origin[tuple(self.agents.origin)] = self.agents.id        
        
    def rectangularish_occupy(self):
        # iterate over egents and perform the walk
        self.agents.rectangularish_occupy(self)
        # update the agent states in environment
        self.voxelsleft = self.number_of_iterations - self.agents.voxelsoccupied
        self.update_agents_rectangularish()
        #print(self.voxelsleft)
        
    def squarish_occupy(self,iter):
        # iterate over egents and perform the walk
        var = self.agents.squarish_occupy(self,iter)
        self.voxelsleft = self.number_of_iterations - self.agents.voxelsoccupied
        #print(self.voxelsleft)
      
        # update the agent states in environment
        self.update_agents_squarish(var)
        

## 05. Run Agent Based Simulation ##

In [101]:
available_index_list= list(np.argwhere(lattice.flatten()==1).flatten())
# Defining agents
def initialize_agents_random_origin (stencil,avail_lattice,agn_num):
    avail_index = np.array(np.where(avail_lattice == 1))
    avail_index_1d = np.ravel_multi_index(avail_index, avail_lattice.shape)
    select_id = np.random.choice(avail_index_1d, 1)
    agn_origins = np.array(np.unravel_index(select_id[0],avail_lattice.shape)).T
    
    myagent = agent(agn_origins, stencil, agn_num)
    return myagent

def initialize_agents_defined_origin (stencil,avail_lattice,agn_num,origin_id):
    agn_origins = np.array(np.unravel_index(origin_id,avail_lattice.shape)).T
    myagent = agent(agn_origins, stencil, agn_num)
    return myagent


In [102]:
origin_list_NE = [3208,3094,3652,3649,3760,3427,3757,3094,3868,3082]
origin_list_NW = [658,988,1093,760,1432,1084,406,736,733,1069]
origin_list_SE = [3043,3703,3901,3010,2458,2674,3817,2485,2566,3484]
origin_list_SW = [709,592,1489,793,1009,1561,1807,1810,1597,2371]
all_combinations = list(itertools.product(origin_list_SE, origin_list_NE, origin_list_NW, origin_list_SW))


# Number of permutations you want to generate
num_permutations = 50  # Example: Limit to 10 permutations
permutations = random.sample(all_combinations, min(num_permutations, len(all_combinations)))
# Shuffle numbers within each selected permutation
shuffled_permutations = [random.sample(perm, len(perm)) for perm in permutations]

avail_lattice = lattice*1


In [103]:
indices = np.argwhere(lattice.flatten() == 1).flatten()
centers = lattice.centroids
print(len(centers))
# initiating the plotter
# p = pv.Plotter(notebook=True) # ITK plotter for interactivity within the python notebook (itkwidgets library is required)
# indices_list = np.concatenate([np.where(indices == num)[0] for num in Origin_List]).tolist()
# print(indices_list)

# # fast visualization of the point cloud
# #sample_cloud.fast_notebook_vis(p)

# # fast visualization of the lattice
# lattice.fast_notebook_vis(p)

# # adding the base mesh: light blue
# mesh = pv.read(mesh_path)
# #p.add_mesh(mesh, color='#abd8ff')

# # adding the ray origins: dark blue
# #p.add_points(pv.PolyData(ray_origins), color='#004887')
# #p.add_points(centers, color='#ff5733')

# p.add_point_labels(centers[indices_list], indices[indices_list], font_size=10, point_size=5)
# p.view_xy()
# #p.screenshot("high_res_image.png", window_size=[4000, 3000])
# # plotting
# p.show()


961


## Running the simulation where you can see each step of the agent 

In [104]:
#Runs it like an animation with all agents together 


env_availability_viz = []

for a in range(num_permutations):
    Agent_Z1=initialize_agents_defined_origin (stencil_squareness_t,lattice*1,10,permutations[a][0]) 
    Agent_Z2=initialize_agents_defined_origin (stencil_squareness_t,lattice*1,20,permutations[a][1])
    Agent_Z3=initialize_agents_defined_origin (stencil_squareness_t,lattice*1,30,permutations[a][2])
    Agent_Z4=initialize_agents_defined_origin (stencil_squareness_t,lattice*1,40,permutations[a][3])

    occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice), avail_lattice)
    env_P_1= {"availibility": occ_lattice_sim,"enviornment": Z1_Desirability_lattice}
    env_P_2 = {"availibility": occ_lattice_sim,"enviornment": Z2_Desirability_lattice}
    env_P_3 = {"availibility": occ_lattice_sim,"enviornment": Z3_Desirability_lattice}
    env_P_4 = {"availibility": occ_lattice_sim,"enviornment": Z4_Desirability_lattice}

    env_1 = environment(env_P_1,Agent_Z1,200,"rectangularish_occupy")
    env_2 = environment(env_P_2,Agent_Z2,150,"rectangularish_occupy")
    env_3 = environment(env_P_3,Agent_Z3,385,"rectangularish_occupy")
    env_4 = environment(env_P_4,Agent_Z4,225,"rectangularish_occupy")


    env_list =[env_1,env_2,env_3,env_4]
    number_steps = max(map(lambda e:e.number_of_iterations,env_list))
    #print(number_steps),env_2,env_3,env_4,env_5,env_6,env_7
        
    for e in env_list:
        a = 0
        while e.voxelsleft > 1 :
            if e.method_name == "rectangularish_occupy":
                e.rectangularish_occupy()
                
            elif e.method_name == "squarish_occupy" :
                e.squarish_occupy(a)               
            a = a+1 
    env_availability_viz.append(e.availibility-1)



In [105]:
# #Runs it like an animation with all agents together 
# env_availability_viz = []

# env_list =[env_1,env_2,env_3,env_4,env_5,env_6,env_7,env_8,env_9,env_10,env_11,env_12,env_13,env_14,env_15,env_16,env_17]
# number_steps = max(map(lambda e:e.number_of_iterations,env_list))
# #print(number_steps),env_2,env_3,env_4,env_5,env_6,env_7
    
# for e in env_list:
#     a = 0
#     while e.voxelsleft > 1 :
#         if e.method_name == "rectangularish_occupy":
#             e.rectangularish_occupy()
            
#         elif e.method_name == "squarish_occupy" :
#             e.squarish_occupy(a)               
#         a = a+1 
# env_availability_viz.append(e.availibility-1)

In [106]:
# Sequential_env_availability_viz = []

# env_list = [env_1, env_2, env_3, env_4]

# # Iterate over each environment one after the other
# for e in env_list:
#     for a in range(e.number_of_iterations):
#         # Execute the respective method based on the environment's method_name
#         if e.method_name == "random_occupy_squareness":
#             e.random_occupy_squareness()
        
#         elif e.method_name == "argmax_occupy_von_neumann":
#             e.argmax_occupy_von_neumann()
        
#         elif e.method_name == "one_neighbour_occupy_squareness_moore":
#             e.one_neighbour_occupy_squareness_moore()
        
#         elif e.method_name == "new_method":
#             e.new_method(a)

#     # Store availability only after the environment has completed all its iterations
#     Sequential_env_availability_viz.append(e.availibility - 1)

In [107]:
p = pv.Plotter(notebook=True)

base_lattice = avail_lattice

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
# Create the spatial reference
grid = pv.ImageData()

# Set the grid envelope_lattice: shape because we want to inject our values
nx, ny, nz = base_lattice.shape
grid.dimensions = (nx + 1, ny + 1, nz + 1)  # Dimensions = cells + 1 for points
# The bottom left corner of the data set
grid.origin = base_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit
# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = env_availability_viz[f]

    # Add the data values to the cell data
    grid.cell_data["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.1, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = sum(map(lambda e:e.number_of_iterations,env_list))
print(number_steps_2)
p.add_slider_widget(create_mesh, [-1, 200], title='Time', value=0, style="classic")
p.view_xy()

p.show()

960


Widget(value='<iframe src="http://localhost:53400/index.html?ui=P_0x1878c513c50_9&reconnect=auto" class="pyvis…

In [109]:
for i, lattice in enumerate(env_availability_viz):
    csv_path = os.path.relpath(f'../Case_Simulation/Financial_Center/4_Agents/Zoning_Options_f_{i:03}.csv')
    lattice.to_csv(csv_path)